# Supervised Pixel Classifiers

This notebook depends on the main pipeline image analysis steps having been done first!

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn.objects as so

import tifffile as tf

import palmettobug as pbug

The PalmettoBUG package is copyrighted 2024-2025 by the Medical University of South Carolina and licensed under the GPL-3 license.
It is free & open source software, can  be redistributed in compliance with the GPL3 license, and comes with absolutely no warranty.
In python, use palmettobug.print_license() to see the license, or use palmettobug.print_3rd_party_license_info() to print information
about the licenses and copyright of 3rd party software used in PalmettoBUG itself or in the creation of PalmettoBUG.


### CHANGE The following directory to match an existing directory on your computer if you are testing this tutorial on your own machine!

In [2]:
my_computer_path =  "C:/Users/Default/Desktop"  ## CHANGE This DIRECTORY to match an existing directory on your computer if you testing this tutorial on your own machine!

## Setup a Supervised Pixel Classifier


A variety of parameters must be decided when setting up a supervised classifier:

    1). The classifier name
    
    2). directories
            a. the directory where the pixel classifier will be setup in (it will contain the classifier information, output, etc.)
            b. the directory of images from which the classificaiton will be run
            
    3). The number of classes to be predicted (and ideally, a dictionary matching each class # with a biological  label).
            - The first class is always 'background', and gets treated differently by default
            - The class dictionary is optional at first (as training, prediction, etc. can proceed without it), but will be needed later
                if you want meaningful labels to be assigned to each class. Otherwise, the classes will only be represented as numbers.
                
    4). A dictionary of the channels to use in the classification
            - only channels in the dictionary will be used!
            - format of dictionary = {'Channel_name': channel_integer}
            - The key part for successful training / prediction is the integer (not the channel name), as whatever
               integers are provided as values in this dictionary will be the channels of the image used in classification.
               The channel names are useful because they are saved in a .json file and provide a record of what antigens
               were intended to be used. However, the channel names are not used in training / prediction itself.

    5). Features to use from the channels
            - These are transformations of the channel data that occur before they are provided to the classifier
            - For example, the 'GAUSSIAN' feature is the simplest available feature and involves a gaussian blurring
              of the channels before being supplied to the classifier. NOTE: there is no 'no transformation' feature and the 
              untransformed values of the channels are not passed into the classifier -- because of this, the 'GAUSSIAN' feature
              is the feature that represents the intensities of the channel and is essentially always used. 
            - Other features can bring out details, such as edges, gradients, etc. in the channels' signal, but do not represent simple
              channel intensity. For a full examination of what these features are, see QuPath's documentation about pixel classification
              as it is more thorough & QuPath is where the features offered in palmettobug are derived from.
               
    6). The Sigma(s) for blurring channels
            - These determine how much blurring is applied before / while deriving features for a channel. Smaller sigmas mean smaller blurring kernels, which in
              turn means more locally determined values for the features, whereas larger sigmas means larger blurring kernels and more contribution of non-local / distant 
              pixels to each channel's features.
            - More than one sigma can be used at once, which can be useful in order to provide the classifier information about different scales in the image.
            - The final number of information layers passed to the classifier is equal to (the number of sigmas) * (the number of channels) * (the number of features)
              This is because all possible combinations of sigmas and features are applied to every selected channel. This contrasts with Unsupervised Classifiers.

Other, less commonly needed parameters -- such as training rate, internal architecture for the classifier, or whether to return categorial classification maps -- can also be set at this step
but can often be ignored. Of these, the most likely to edit would be the internal architecture of the neural network used as a classifier. This is supplied as a list of integers that 
determine how many neurons to place in the hidden layers of the network. By default, the classifier's network has no hidden layers.

In [3]:
''' name and directories '''

my_classifier_name = "lumen_epithelia_laminapropria.json"

pixel_directory = f"{my_computer_path}/Example_IMC"
pixel_class_object = pbug.SupervisedClassifier(homedir = pixel_directory) ## the pixel classifier object can be created with only the set-up directory,
                                                                          ## but before training can be proceed we will need to devided more parameters

image_directory = pixel_directory + "/images/img"      ## the source of the images you want to classify train / predict from

In [4]:
''' Select classes & setup dictionary '''

classes = ["background", "epithelia", "lamina_propria"]      ## "background" should always be the first class -- in this case it wil also be the stand-in for 'lumen'
classes_dictionary = {1:"background",2:"epithelia",3:"lamina_propria"}      ### when the classifier exports a classification prediction, and when training labels are made in Napari, it will be in the form of integer labels
                                                                        ## this dictionary identifies the correspondence betwen integer labels, and the biological labels
                                                                        ## WARNING! 'background' should ALWAYS BE THE FIRST CLASS! This is because class 1 is automatically treated differently by palmettobug
number_of_classes = len(classes)

In [5]:
''' Channel Dictionary 

Here I select all channels using the panel + a loop, but otherwise this should be set manually with the format as in the following example:
        {'Vimentin': 2, 'Vitronectin': 19, 'Beta-Catenin': 26}

Where the number for each channel corresponds to that channel's order in the images (zero-indexed / as it would be displayed in napari), or 
according to that channel's index in the panel file (after dropping unwanted channels & reseting the index)
'''
panel = pd.read_csv(f"{pixel_directory}/panel.csv")
panel = panel[panel['keep'] == 1].reset_index()           ########## drop & reset index to use the panel file!

channel_dictionary = {}  
for i,ii in zip(panel.index, panel['name']):
    channel_dictionary[ii] = i

channel_dictionary

{'aSMA': 0,
 'p-selectin': 1,
 'Vimentin': 2,
 'CD14': 3,
 'CD31': 4,
 'CD16': 5,
 'Pan-Keratin': 6,
 'CD11b': 7,
 'CD163': 8,
 'CD45': 9,
 'CD206': 10,
 'CCL2': 11,
 'FoxP3': 12,
 'CD4': 13,
 'E-cadherin': 14,
 'CD68': 15,
 'CD66b': 16,
 'CD20': 17,
 'CD8': 18,
 'Vitrionectin': 19,
 'CD32b': 20,
 'GranzymeB': 21,
 'Ki-67': 22,
 'Collagen-1': 23,
 'CD3': 24,
 'HistoneH3': 25,
 'Beta-Catenin': 26,
 'CD45RO': 27,
 'HLA-DR': 28,
 'DNA1': 29,
 'DNA2': 30,
 'Seg1': 31,
 'Seg2': 32,
 'Seg3': 33}

In [6]:
''' Selects sigma(s) and feature(s) '''

sigma_list = [1.0, 2.0, 4.0]         ## a list of floats, denotes the sigmas to use for features --> see QuPath supervised pixel classifiers for details of what this means

possible_features =  ["GAUSSIAN", "LAPLACIAN", "WEIGHTED_STD_DEV", "GRADIENT_MAGNITUDE", "STRUCTURE_TENSOR_EIGENVALUE_MAX", 
                      "STRUCTURE_TENSOR_EIGENVALUE_MIN", "STRUCTURE_TENSOR_COHERENCE", "HESSIAN_DETERMINANT", 
                      "HESSIAN_EIGENVALUE_MAX",  "HESSIAN_EIGENVALUE_MIN"]

features_list = ["GAUSSIAN"]                 ## fewer features is often better, see QuPath pixel classification documentation for details. Each choesn feature is applied to ALL channels
                                                                     # GAUSSIAN is almost always recommended  (it is the blurred representation of the selected channels)

In [7]:
internal_architecture = []         ### or a list of integers indicating te sizes of hidden layers in the ANN_MLP. Such as [16,8]. 

## this function writes the dictionary / .json file to the classifier directory and sets up the ANN_MLP for training:
_ = pixel_class_object.setup_classifier(classifier_name = my_classifier_name, number_of_classes = number_of_classes, 
                            sigma_list = sigma_list, features_list = features_list, 
                            channel_dictionary = channel_dictionary, image_directory = image_directory, classes_dictionary = classes_dictionary,  
                            categorical = True, internal_architecture = internal_architecture, epsilon = 0.01, iterations = 1000)

## Creating Training labels

Once the classifier is setup, we must manually label parts of our images as each of the classes (including background) in the classifier.
This can be conveniently done inside Napari -- although that means that this tutorial cannot directly show how that is done, as the labeling
steps happens outside the notebook.

Here, we launch one of source images in napari, along with a label layer that we can annotate inside nappari to represent our training labels.
Once the labels are completed, we can close napari and run the second cell that writes our labels to a folder inside the classifier's directory.

In [9]:
'''NOTE! The training labels used in this example notebook can be found in the /training_labels subfolder where this notebook is!

If you want to exactly reproduce the results of these notebooks on your machine, move that labeling image into your pixel classifier by uncommenting the 4 lines below,
and then be sure to not edit the labeling in the following two cells (running those two cells can still be useful to see the labeling layer, but ANY edits will ruin reproducibility of the classifier):
'''

import shutil
#shutil.rmtree(pixel_class_object.classifier_training_labels)
#shutil.copytree(os.getcwd() + "/training_labels", pixel_class_object.classifier_training_labels)   ## be sure your current working directory is where this notebook is saved!

display()

############### Only run this cell once!

In [ ]:
'''
Make training labels (done inside napari, this code only launches napari) 

Repeat this code as many times as desired for the various images in your dataset, until you have the number of annotated images that you want

Remember your class dictionary pairings: {1:"background", 2:"epithelia", 3:"lamina_propria"}  while making labels!

If you have previously made a set of training labels for an image, and then re-open that image using this code, napari should automatically
load your prior labels, allowing you examine, update, and edit those prior labels. 
'''
image_number = 0

image_paths =["".join([image_directory,"/",i]) for i in os.listdir(image_directory)]

import warnings
warnings.filterwarnings("ignore", message = "pyside_type_init")
pixel_class_object.launch_Napari_px(image_path = image_paths[image_number], display_all_channels = False)        ### display_all_channels changes the axis of the napari display

In [11]:
pixel_class_object.write_from_Napari()   ## this function is necessary to write the labels made in napari after napari has been closed. --> it must be in a separate cell

###  WARNING!    this needs to be run afterwards in a separate cell as the Napari launch!

Training labels written!


## Training & Prediction

Once training labels exist, training and prediction are extremely simple, only requiring that you pass in the source directory for images (which should be the 
same as the image directory you had previously chosen).

During training, all training labels in the classifer's training_labels directory are used -- This means that if you want to discard previously saved traning labels,
you must edit them in napari, or manually delete them from the classifier's training_labels directory. 

NOTE!: Normally, you would want to make labels for multiple different images, creating training lables from diverse images and diverse areas of the images. 
Doing this is important to help improve the classifier's performance across your dataset -- otherwise it may only be competent at classifying pixels from images that look like the image(s) 
you made training labels for. However, because this is just demonstration code, though, I will not bother with making multiple rounds of training labels.

In [12]:
'''
Now, Train & Predict with the classifier
'''
_ = pixel_class_object.train_folder(image_directory)

In [13]:
'''
Now generate pixel classification maps from our training classifier:
'''
pixel_class_object.predict_folder(image_directory)     ## if specificied, the output_directory for the predictions can be set to a differnet location than the default folder inside the classifier's directory

In [14]:
'''Now, let's look at one of our predictions:'''

prediction_paths = ["".join([pixel_class_object.output_directory,"/",i]) for i in os.listdir(pixel_class_object.output_directory)]  

image_number = 7   ## image 0 looks decent because that is the only image where I made my training labels, but for image 7 (and others) the pixel class predictions look quite poor!  
                   ## If your predictions look poor, then make addiitonal training labels for the cases where the classifier had been failing, then repeat training & prediciton

pbug.run_napari(image = tf.imread(image_paths[image_number]), masks = tf.imread(prediction_paths[image_number]))

## What can you do with these Classifier Predictions?

## 1). crop / slice images to mimum area eoncompassing the class of interest

You can crop images down to only be the regions where you class of interest is
Or: set background / undesirable / noisy pixel to zero

In [15]:
class_to_keep = [2]    ## remmber that class 2 was epithelia
class_map_folder = pixel_class_object.output_directory
image_folder = image_directory
output_folder = pixel_directory + "/images/sliced_by_epithelia"
padding = 0                                                                            ## whether to keep a layer of pixels around the class
zero_out = False                                                                       ## whether to additionally set all channels in the non-class pixels to 0


pbug.slice_folder(class_to_keep = class_to_keep,
                   class_map_folder = class_map_folder, 
                   image_folder = image_folder, 
                   output_folder = output_folder,
                   padding = padding, zero_out = zero_out)

Key not found: list index out of range
Key not found: list index out of range
key not found list index out of range
Key not found: list index out of range
Key not found: list index out of range
Key not found: list index out of range
key not found list index out of range
Key not found: list index out of range
Key not found: list index out of range
Key not found: list index out of range
key not found list index out of range
Key not found: list index out of range
Key not found: list index out of range
Key not found: list index out of range
key not found list index out of range
Key not found: list index out of range
Key not found: list index out of range
Key not found: list index out of range
key not found list index out of range
Key not found: list index out of range
Key not found: list index out of range
Key not found: list index out of range
key not found list index out of range
Key not found: list index out of range
Key not found: list index out of range
Key not found: list index out o

## 2). Directly Segment objects using the classifier

This generally depends on the objects you want to segment being of similar size, being round / convex, and not overlapping too much.
For example, this style of segmentation can be effective for nuclei.

In [ ]:
'''
Direct segmentation from pixel classifier:
The class that is used for segmentation is split into masks by calculating distance-from-background, finding the maximum points, and using those points as seeds for a watershedding algorithm
This requires that 'cells' / masking regions be well-separated and have distinct break points between cells

A more usual use case might be to classify nuclei from background to get nuclei masks -- which tend to be round and more easily separable by this method --
then apply an expansion (whether by merging with a pixel classifier's output, or by using a simple/fixed pixel expansion) to try to capture the cytoplasm around the nuclei.
'''
pixel_classifier_directory = pixel_class_object.output_directory
output_folder = pixel_directory + "/masks/lumen_segmentation"
threshold = 25                   ### masks smaller that this threshold will ber prohibited
distance_between_centroids = 45  ## changing this parameter determines how close the centroids of the masks can be during watershedding, for increasing it favors larger, fewer masks while decreasing it favors many smaller masks.
                                ## Increasing it can be very useful to curb excessive fragmentation of masks
background = 3
'''Here we 'segment' the lumens of crypts in the image. Why? Becuase of the objects in this particular classifier, the lumen are the most circular, and therefore will likely be the best behaved when segmenting this way.'''
to_segment_on = [1]             ## a list, by integer, of the classes to include in the segmentation (in this case, crypt lumen)

pbug.segment_class_map_folder(pixel_classifier_directory = pixel_classifier_directory, 
                                  output_folder = output_folder, 
                                  distance_between_centroids = distance_between_centroids,
                                  threshold = threshold, 
                                  to_segment_on = to_segment_on, 
                                  background = background)    ## as currently set up, this only work on the final class in the images, after background is removed / zero'd out)

## 3). Classify segmented cells using the clasifier

We can use a classifer to cluster cells into groups, by looking at what pixel classes are inside the segmenttaion regions.
Here, we will classify the cells as belonging to either lumen, epithelia, or lamina_propia by looking at what is the most common pixel class 
within each cell (as in, the mode). 

In [17]:
'''
Now, classify the deepcell masks using the classification maps we just generated
'''
name = "My_classy_deepcell_masks"

mesmer_mask_folder = pixel_directory + "/masks/example_deepcell_masks"     ## or "/masks/deepcell"  
classifier_mask_folder = pixel_class_object.output_directory
run_folder = pixel_directory + f"/classy_masks/{name}"
output_folder = run_folder + f"/{name}"   

if not os.path.exists(run_folder):
    os.mkdir(run_folder)

mask_classifications = pbug.mode_classify_folder(mesmer_mask_folder, classifier_mask_folder, output_folder, merging_table = None)

## to save the mask_classifications for a potential later import into a CATALYST-style analysis:
mask_classifications.to_csv(run_folder + f"/{name}.csv", index = False)   
                                                            ## This precise naming is chosen for compatibility with loading into the GUI later -- however if you never intend
                                                            ## to use the GUI, then any name for this will do.

In [18]:
'''The classifications are stored as numbers, but these can decoded back into biological labels using the 'biological_labels' csv in the pixel classifier.'''

display(mask_classifications)
pd.read_csv(pixel_class_object.classifier_dir + "/biological_labels.csv")

,classification
0,3
1,2
2,3
3,2
4,2
...,...
4665,3
4666,3
4667,3
4668,3


,Unnamed: 0,class,labels,merging
0,0,1,background,0
1,1,2,epithelia,2
2,2,3,lamina_propria,3


## 4). Extend masks into matching pixel classification

Existing cell segmentation masks can be expanded by pixel class. This can be particularly useful in cases where the segmentation algorithms
are mainly finidng circular, simple shapes, but the expected biological cell shapes are highly irregular / branched. 
Example: astrocyte segmentation can benefit from making an astrocyte/GFAP pixel class, classifying cells as astrocyte from that, then 
        extending those astrocyte cells into the surrounding astrocyte pixel class -- "decorating" the circular astorcyte cell bodies
        identified by cell segmentation with the irregular cytoplasmic projections captured by the pixel classifier.

Specifically:

    First, you MUST have an existing cell segmentation and have classified that set of cell segmentation masks (see use #3 above). 
        "Classy masks" are necessary so that cell not of the pixel class of interest will not be extended.
    
    Second, you must select which classes to extend masks into. Often, this is a very specific class (for example, GFAP pixels for astrocytes), and usually
        you don't want to extend into every class (especially background), as this tends to extend cells into massive, non-biological shapes. For example, 
        in this case extending cells into the 'lumen class' will mean that the rare cells in the 'lumen' regions (which we expect to be largely empty / acellular) 
        could be expanded to fill the entire 'lumen' region that they inside of.
    
    Next, inside the mask extending function the classes of interest are iterated over, with each cell mask belonging to the class being expanded by
        watershedding (from the centroid of the cell mask) into any pixels of that class that touch the mask. The watershedding is done simultaneously
        for all masks of a given class at once, so that final boundaries for multiple cells matching cells in contact with a pixel class region is determined 
        by the watershedding (the original cell masks protected from losing pixels to a neighboring cell in this process).

In [19]:
'''
Now merge the masks and pixel classifier together, with the help of the classy masks!
'''
classy_mesmer_mask_folder = output_folder     ## from classy masksing above
output_directory_folder = pixel_directory + "/masks/extended_masks"

connectivity = 2   ## 1 or 2, an argument in skimage's watershedding function -- determines if diagonally adjacent pixels are considered adjacent for the watershedding
merge_list = [2,3]   ## the classes to merge --> in this case epithelia and lamina_propria

pbug.extend_masks_folder(classifier_mask_folder, mesmer_mask_folder, 
                        classy_mesmer_mask_folder, output_directory_folder,
                        merge_list = merge_list, 
                        connectivity = connectivity)